In [ ]:
var dumpFilePath = await Kernel.GetInputAsync(
    "Please provide a memory dump file.",
    typeHint: "file");

In [ ]:
#r "nuget: Microsoft.Diagnostics.Runtime, 3.0.0-beta.23214.4"

using Microsoft.Diagnostics.Runtime;
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Formatting;
using System;

Formatter.Register<ulong>(n => n.ToString("x16"), "text/html");
Formatter.ListExpansionLimit = int.MaxValue;

In [ ]:
using (var dataTarget = DataTarget.LoadDump(dumpFilePath))
{
    foreach (var version in dataTarget.ClrVersions)
    {
        var clrRuntime = version.CreateRuntime();

        if (!clrRuntime.Heap.CanWalkHeap)
        {
            Console.WriteLine("Cannot walk the heap!");
            continue;
        }

        var sqlCommandRoots = (from clrObject in clrRuntime.Heap.EnumerateObjects()
                        where clrObject.IsValid && !clrObject.IsFree || clrObject.Type.Name != "System.Data.SqlClient.SqlCommand"
                        from referenceClrObject in clrObject.EnumerateReferencesWithFields()
                        where referenceClrObject.Object.Type.Name == "System.Data.SqlClient.SqlCommand"
                        select new
                        {
                            SqlCommand = referenceClrObject.Object.Address,
                            Root = new
                            {
                                Address = clrObject.Address,
                                Type = clrObject.Type.Name,
                                Field = referenceClrObject.Field.Name,
                            }
                        }).ToLookup(r => r.SqlCommand, r => r.Root);

		var sqlCommands = from clrObject in clrRuntime.Heap.EnumerateObjects()
						  where clrObject.IsValid && !clrObject.IsFree && clrObject.Type.Name == "System.Data.SqlClient.SqlCommand"
						  select new
						  {
							  Address = clrObject.Address,
							  ConnectionAddress = clrObject.ReadObjectField("_activeConnection").Address,
							  CommandText = clrObject.ReadStringField("_commandText"),
							  Roots = sqlCommandRoots[clrObject.Address].ToList(),
						  };

        sqlCommands.Display();
    }
}
